<a href="https://colab.research.google.com/github/vichitrarora/warp/blob/main/Another_copy_of_Unsloth_SFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 2.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-0.5B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = True,

)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/457M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.71k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [2]:
loftq_config = {"bits": 4}
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 64,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,
    loftq_config = loftq_config, # And LoftQ
)

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/config.py:455: UserWarning: `loftq_config` specified but will be ignored when `init_lora_weights` is not 'loftq'.
  warnings.warn("`loftq_config` specified but will be ignored when `init_lora_weights` is not 'loftq'.")
Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.2.15 patched 24 layers with 24 QKV layers, 24 O layers and 0 MLP layers.


In [4]:
import pandas as pd
import json


file_path = "/content/train_data_v4.csv"
df = pd.read_csv(file_path)


dataset = df.drop_duplicates()

dataset["mongo_query"] = dataset["mongo_query"].str.replace("'", '"')


dataset["mongo_query"] = dataset["mongo_query"].str.rstrip(";")


def is_valid_json(text):
    try:
        json.loads(text)
        return True
    except:
        return False

dataset = dataset[dataset["schema"].apply(is_valid_json)]


dataset.to_csv("train_data_v4_cleaned.csv", index=False)


print("Cleaning complete. Saved as 'train_data_v4_cleaned.csv'.")
dataset.head()


Cleaning complete. Saved as 'train_data_v4_cleaned.csv'.


<ipython-input-4-bb4014fe6ea4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["mongo_query"] = dataset["mongo_query"].str.replace("'", '"')
<ipython-input-4-bb4014fe6ea4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["mongo_query"] = dataset["mongo_query"].str.rstrip(";")


,database_id,schema,mongo_query,natural_language_query
0,traffic_event,"{\n ""collections"": [{\n ""name"": ""traffic_e...",db.traffic_events.find({}),Retrieve all traffic events with details about...
1,traffic_event,"{\n ""collections"": [{\n ""name"": ""traffic_e...",db.traffic_events.find({}),Retrieve all recorded traffic events with thei...
2,traffic_event,"{\n ""collections"": [{\n ""name"": ""traffic_e...","db.traffic_events.find({ ""identifier.intersect...",Retrieve all traffic event records for the int...
3,traffic_event,"{\n ""collections"": [{\n ""name"": ""traffic_e...","db.traffic_events.find({ ""identifier.intersect...",Retrieve all traffic event records for the int...
4,traffic_event,"{\n ""collections"": [{\n ""name"": ""traffic_e...","db.traffic_events.find({ ""response.event.sever...",Retrieve all traffic events with critical seve...


In [5]:
model_name = "unsloth/Qwen2.5-0.5B-bnb-4bit"

In [6]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer


file_path = "/content/train_data_v4_cleaned.csv"
df = pd.read_csv(file_path)


df.rename(columns={
    "database_id": "db_id",
    "schema": "schema_info",
    "natural_language_query": "nl_query",
    "mongo_query": "mongo_query"
}, inplace=True)


df["instruction"] = "Generate a MongoDB query from the given database schema and natural language request."
df["input"] = df.apply(lambda row: f"Database ID: {row['db_id']}\nSchema: {row['schema_info']}\nQuery: {row['nl_query']}", axis=1)
df["output"] = df["mongo_query"]


tokenizer = AutoTokenizer.from_pretrained(model_name)


prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token


def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []

    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = prompt.format(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}


dataset = Dataset.from_pandas(df)


dataset = dataset.map(formatting_prompts_func, batched=True)

print(dataset[0]["text"])


tokenizer_config.json:   0%|          | 0.00/4.71k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

Map:   0%|          | 0/1431 [00:00<?, ? examples/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a MongoDB query from the given database schema and natural language request.

### Input:
Database ID: traffic_event
Schema: {
  "collections": [{
    "name": "traffic_events",
    "document": {
      "properties": {
        "identifier": {
          "bsonType": "object",
          "properties": {
            "intersection_id": {"description": "Filter by traffic junction"},
            "camera_id": {"description": "Filter by surveillance unit"}
          }
        },
        "response": {
          "bsonType": "object",
          "properties": {
            "event": {
              "properties": {
                "severity": {"description": "Critical, High, Low"},
                "type": {"description": "Speeding, RedLightViolation, IllegalParking"},
                "vehicles": {
                  "bsonTy

In [7]:
from datasets import load_dataset  # Import this!

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

sft_data = load_dataset("csv", data_files={"train": "train_data_v4.csv"})
EOS_TOKEN = tokenizer.eos_token
def preprocess_sft(example):
    input_text = f"Generate a MongoDB query for the following user input.\n UserInput: {example['natural_language_query']}\n The name of the database is {example['database_id']} having schema {example['schema']}."
    target_text = str(example["mongo_query"])
    prompt = f"### Instruction:Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n{input_text}\n\n### MongoDB Query:\n{target_text}\n"+EOS_TOKEN
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
    return tokenized

tokenized_sft = sft_data.map(preprocess_sft, batched=False)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1617 [00:00<?, ? examples/s]

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
sft_training_args = TrainingArguments(
    output_dir="sft_output",
    evaluation_strategy="steps",
    eval_steps=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=200,
    report_to="none",
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
from trl import SFTTrainer
sft_trainer = SFTTrainer(
    model=model,
    args=sft_training_args,
    evaluation_strategy="no",
    do_eval=False,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML (num_proc=2):   0%|          | 0/1431 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/1431 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/1431 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/1431 [00:00<?, ? examples/s]

In [10]:
sft_trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,431 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 1
\        /    Total batch size = 8 | Total steps = 537
 "-____-"     Number of trainable parameters = 4,325,376


Step,Training Loss
50,0.301300
100,0.094800
150,0.083000
200,0.077900
250,0.071800
300,0.067800
350,0.066000
400,0.061300
450,0.057500
500,0.059200


TrainOutput(global_step=537, training_loss=0.09157988656610527, metrics={'train_runtime': 761.3411, 'train_samples_per_second': 5.639, 'train_steps_per_second': 0.705, 'total_flos': 7942663502036736.0, 'train_loss': 0.09157988656610527})

In [11]:
from peft import PeftModel
import torch

# Define the save directory
lora_save_directory = "./lora_trained_model"

# Save only LoRA adapters (not full model)
model.save_pretrained(lora_save_directory)
tokenizer.save_pretrained(lora_save_directory)

print(f"LoRA adapters saved at {lora_save_directory}")


LoRA adapters saved at ./lora_trained_model


In [12]:
import os

lora_save_directory = "./lora_trained_model"

# List saved files
print("Files in LoRA save directory:")
print(os.listdir(lora_save_directory))


Files in LoRA save directory:
['vocab.json', 'merges.txt', 'adapter_config.json', 'tokenizer.json', 'special_tokens_map.json', 'tokenizer_config.json', 'added_tokens.json', 'README.md', 'adapter_model.safetensors']


In [13]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
from peft import PeftModel

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define paths
base_model_name = "unsloth/Qwen2.5-0.5B-bnb-4bit"  # Base model
lora_model_path = "./lora_trained_model"  # LoRA adapters (your saved directory)

# Load tokenizer from LoRA adapter directory
tokenizer = AutoTokenizer.from_pretrained(lora_model_path)

# Load base model
model, _ = FastLanguageModel.from_pretrained(base_model_name, load_in_4bit=True)
model.to(device)

# Load LoRA adapters
model = PeftModel.from_pretrained(model, lora_model_path)

# Optimize for inference
FastLanguageModel.for_inference(model)

print("✅ LoRA fine-tuned model loaded successfully!")


==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ LoRA fine-tuned model loaded successfully!


In [14]:
for name, param in model.named_parameters():
    if "lora" in name.lower():
        print(f"LoRA Parameter Found: {name} | Trainable: {param.requires_grad}")


LoRA Parameter Found: base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight | Trainable: False
LoRA Parameter Found: base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight | Trainable: False
LoRA Parameter Found: base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight | Trainable: False
LoRA Parameter Found: base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight | Trainable: False
LoRA Parameter Found: base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight | Trainable: False
LoRA Parameter Found: base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight | Trainable: False
LoRA Parameter Found: base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight | Trainable: False
LoRA Parameter Found: base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight | Trainable: False
LoRA Parameter Found: base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight | Trainable

In [15]:
from peft import get_peft_model_state_dict

lora_state_dict = get_peft_model_state_dict(model)
if len(lora_state_dict) > 0:
    print("✅ LoRA adapters are correctly loaded!")
else:
    print("❌ LoRA adapters are NOT loaded correctly. Check the save path.")


✅ LoRA adapters are correctly loaded!


In [16]:
def generate_mongo_query(nl_query, db_id, schema_info):
    model.eval()  # Set to inference mode

    prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a MongoDB query from the given database schema and natural language request.

### Input:
Database ID: {db_id}
Schema: {schema_info}
Query: {nl_query}

### Response:
"""

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate output
    with torch.no_grad():
        output_ids = model.generate(**inputs, max_length=512)

    # Decode output
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return generated_text

# Example Test
nl_query = "List all traffic events with non critical severity."
db_id = "traffic_event"
schema_info = '"{ ""collections"": [{ ""name"": ""traffic_events"", ""document"": { ""properties"": { ""identifier"": { ""bsonType"": ""object"", ""properties"": { ""intersection_id"": {""description"": ""Filter by traffic junction""}, ""camera_id"": {""description"": ""Filter by surveillance unit""} } }, ""response"": { ""bsonType"": ""object"", ""properties"": { ""event"": { ""properties"": { ""severity"": {""description"": ""Critical, High, Low""}, ""type"": {""description"": ""Speeding, RedLightViolation, IllegalParking""}, ""vehicles"": { ""bsonType"": ""array"", ""items"": { ""properties"": { ""number_plate"": { ""bsonType"": ""string"", ""pattern"": ""^[A-Z]{2}[0-9]{2}[A-Z]{2}[0-9]{4}$"" }, ""speed"": {""bsonType"": ""double""}, ""timestamp"": {""bsonType"": ""date""}, ""location"": { ""bsonType"": ""array"", ""items"": {""bsonType"": ""double""} } } } } } } } } } } }], ""version"": 2 } "'

print("Generated MongoDB Query:\n", generate_mongo_query(nl_query, db_id, schema_info))


Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated MongoDB Query:
 Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a MongoDB query from the given database schema and natural language request.

### Input:
Database ID: traffic_event
Schema: "{ ""collections"": [{ ""name"": ""traffic_events"", ""document"": { ""properties"": { ""identifier"": { ""bsonType"": ""object"", ""properties"": { ""intersection_id"": {""description"": ""Filter by traffic junction""}, ""camera_id"": {""description"": ""Filter by surveillance unit""} } }, ""response"": { ""bsonType"": ""object"", ""properties"": { ""event"": { ""properties"": { ""severity"": {""description"": ""Critical, High, Low""}, ""type"": {""description"": ""Speeding, RedLightViolation, IllegalParking""}, ""vehicles"": { ""bsonType"": ""array"", ""items"": { ""properties"": { ""number_plate"": { ""bsonType"": ""string"", ""pattern"": ""^[A-Z]{2}[0-9

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import torch
import shutil
import os
from peft import PeftModel
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

# Define paths
base_model_name = "unsloth/Qwen2.5-0.5B-bnb-4bit"  # Base model
lora_model_path = "./lora_trained_model"  # LoRA adapter path
save_path = "/content/drive/MyDrive/merged_model"  # Save location in Google Drive

# Load base model
model, tokenizer = FastLanguageModel.from_pretrained(base_model_name)

# Load LoRA adapters and merge
model = PeftModel.from_pretrained(model, lora_model_path)
model = model.merge_and_unload()  # Merge LoRA with base model

# Save the merged model
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"✅ Merged model saved at {save_path}")


==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


✅ Merged model saved at /content/drive/MyDrive/merged_model


In [19]:
import os
print(os.listdir("/content/drive/MyDrive/merged_model"))


['config.json', 'generation_config.json', 'model.safetensors', 'tokenizer_config.json', 'special_tokens_map.json', 'added_tokens.json', 'vocab.json', 'merges.txt', 'tokenizer.json']


In [20]:
import torch
import shutil
import os
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM
from unsloth import FastLanguageModel

# 🔹 Define paths
base_model_name = "unsloth/Qwen2.5-0.5B-bnb-4bit"  # Base model
lora_model_path = "./lora_trained_model"  # LoRA adapter path
save_path = "/content/drive/MyDrive/merged_model_fp16"  # Save location in Google Drive

# 🔹 Load base model (without UnsLoft wrapper)
model = AutoModelForCausalLM.from_pretrained(base_model_name, torch_dtype=torch.float16)  # Force FP16
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# 🔹 Load LoRA adapters and merge
model = PeftModel.from_pretrained(model, lora_model_path)
model = model.merge_and_unload()  # Merge LoRA with base model

# 🔹 Ensure Google Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

# 🔹 Save the merged model
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"✅ Merged model saved at {save_path}")


config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/457M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Merged model saved at /content/drive/MyDrive/merged_model_fp16


In [21]:
import os
sft_model_dir = "/content/drive/MyDrive/sft_finetuned_model"
os.makedirs(sft_model_dir, exist_ok=True)
model.save_pretrained(sft_model_dir)
tokenizer.save_pretrained(sft_model_dir)

('/content/drive/MyDrive/sft_finetuned_model/tokenizer_config.json',
 '/content/drive/MyDrive/sft_finetuned_model/special_tokens_map.json',
 '/content/drive/MyDrive/sft_finetuned_model/vocab.json',
 '/content/drive/MyDrive/sft_finetuned_model/merges.txt',
 '/content/drive/MyDrive/sft_finetuned_model/added_tokens.json',
 '/content/drive/MyDrive/sft_finetuned_model/tokenizer.json')